In [1]:
%run client_collection.py

In [2]:
%run preprocessing.py

In [3]:
%run km_arima_lstm.py

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.

Using TensorFlow backend.


In [6]:
## warm up upto preprocessing
'''
final=pd.DataFrame()
window=1000
for _ in range(window):
#while True:
    iterations += 1
    string = socket.recv_string()
    sym, bidPrice,bidSize,askPrice,askSize = string.split()
    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))
    dt = datetime.datetime.now()
    df = df.append(pd.DataFrame({'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))
    data=preprocessing_df(df)
    #print(df.tail(1))
    print(data.tail(1))   
'''

"\nfinal=pd.DataFrame()\nwindow=1000\nfor _ in range(window):\n#while True:\n    iterations += 1\n    string = socket.recv_string()\n    sym, bidPrice,bidSize,askPrice,askSize = string.split()\n    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))\n    dt = datetime.datetime.now()\n    df = df.append(pd.DataFrame({'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))\n    data=preprocessing_df(df)\n    #print(df.tail(1))\n    print(data.tail(1))   \n"

In [11]:
len(data)

100

In [8]:
#testing to delete inf and NaN
data=data[1000:2000]

In [9]:
#checking
data.head(2)

,Close,D,U,askPrice,askSize,bidPrice,bidSize,log,logDiff,price,sigma
2017-09-08 20:25:46.256804,246.755,0,1,246.76,19.0,246.75,228.0,5.508396,0.000041,246.750769,0.000008
2017-09-08 20:25:46.267021,246.755,0,1,246.76,22.0,246.75,228.0,5.508396,0.000041,246.750880,0.000006


In [10]:
import pickle
#from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


df=data
# saving linear model
df=df[1:].dropna()
X=df[['askPrice','askSize','bidPrice','bidSize','Close','U','D','sigma']]
y=df[['logDiff']]
regr = linear_model.LinearRegression()
regr_model=regr.fit(X,y)
regr_model = pickle.dumps(regr_model)
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.9) #kernel='linear' #kernel='poly'
svr_model = svr_rbf.fit(X, y)
svr_model = pickle.dumps(svr_model)

# saving logistics and SVC model
df=df[1:].dropna()
X=df[['askPrice','askSize','bidPrice','bidSize','Close','price','sigma']]
y1=df[['U']]
y2=df[['D']]

svm = SVC(kernel='linear')
lm = linear_model.LogisticRegression(C=1e4)
svm_model_up= svm.fit(X,y1)
svm_model_up = pickle.dumps(svm_model_up)
lm_model_up= lm.fit(X, y1)
lm_model_up = pickle.dumps(lm_model_up)
svm_model_dn= svm.fit(X, y2)
svm_model_dn = pickle.dumps(svm_model_dn)
lm_model_dn= lm.fit(X, y2)
lm_model_dn = pickle.dumps(lm_model_dn)


#loading regression model, first save the model
svr_model = pickle.loads(svr_model)
regr_model = pickle.loads(regr_model)

#loading classification model, first save the model
svm_model_up = pickle.loads(svm_model_up)
svm_model_dn = pickle.loads(svm_model_dn)
lm_model_up = pickle.loads(lm_model_up)
lm_model_dn = pickle.loads(lm_model_dn)

def strat_lr(data):
    data=data.tail(60).dropna()
    X=data[['askPrice','askSize','bidPrice','bidSize','Close','U','D','sigma']]
    y=data[['logDiff']]
    predict_regr=regr_model.predict(X)
    predict_svr=svr_model.predict(X)
    dt=data[['Close']]
    dt['predict_regr']=predict_regr
    dt['predict_svr']=predict_svr
        
    #pdf=data
    #pdf['pREG']=np.exp(dt.predict_regr+data.log.shift(59))
    #pdf['pSVR']=np.exp(dt.predict_regr+data.log.shift(59))
    data['pREG']=np.exp(dt.predict_regr+data.log.shift(59))
    data['pSVR']=np.exp(dt.predict_regr+data.log.shift(59))
    return data

def classification_up_dn(data):
    X=data[['askPrice','askSize','bidPrice','bidSize','Close','price','sigma']]
    y1=data[['U']]
    y2=data[['D']]
    pr_df=data.tail(60)
    predict_svm_up=svm_model_up.predict(X.tail(60))
    pr_df['predict_svm_up']=predict_svm_up
    predict_lm_up=lm_model_up.predict(X.tail(60))
    pr_df['predict_lm_up']=predict_lm_up
    predict_svm_dn=svm_model_dn.predict(X.tail(60))
    pr_df['predict_svm_dn']=predict_svm_dn
    predict_lm_dn=lm_model_dn.predict(X.tail(60))
    pr_df['predict_lm_dn']=predict_lm_dn
    pr_df['predict_svm']=pr_df.predict_svm_up+pr_df.predict_svm_dn
    pr_df['predict_lm']=pr_df.predict_lm_up+pr_df.predict_lm_dn
    return pr_df

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [20]:
import json
import plotly_stream as plyst
import plotly.tools as plyt
import plotly.plotly as ply

In [21]:
import json
import datetime
import time
import random
import pandas as pd
import plotly.plotly as py
import plotly.tools as tls 
from plotly.graph_objs import *
import cufflinks

In [16]:
pc = json.load(open('creds/plotly_creds.json', 'r'))

In [17]:
plyt.set_credentials_file(username=pc['username'], api_key=pc['api_key'])

In [18]:
plyst.plotly_stream.set_stream_tokens(pc['stream_ids'])

In [55]:
#!pip install twisted

In [54]:
# solving module 'twisted' has no attribute '__version__'
#!pip install --upgrade pyopenssl

In [22]:
pcreds = json.load(open('creds/plotly_creds.json', 'r'))
py.sign_in(pcreds['username'], pcreds['api_key'])
from autobahn.twisted.websocket import WebSocketClientProtocol, \
                                       WebSocketClientFactory

In [24]:
# plotly preparations

# get stream id from stream id list
stream_ids = pcreds['stream_ids']

# generate Stream object with maximum points 150
stream_0 = Stream(
    token=stream_ids[0],
    maxpoints=150)

# generate Scatter & Data objects
trace0 = Scatter(
    x=[], y=[],
    mode='lines+markers',
    stream=stream_0,
    name='price')

dats = Data([trace0])

In [44]:
# generate figure object
layout = Layout(title=' ')
fig = Figure(data=dats, layout=layout)
unique_url = py.plot(fig, filename='stream_plot', auto_open=False)

print('URL of the streaming plot:\n%s' % unique_url)

s0 = py.Stream(stream_ids[0])
s0.open()
s1 = py.Stream(stream_ids[1])
s1.open()

URL of the streaming plot:
https://plot.ly/~parthasen/16


### testing

In [67]:
window=10
for _ in range(window):
    iterations += 1
    string = socket.recv_string()
    sym, bidPrice,bidSize,askPrice,askSize = string.split()
    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))
    dt = datetime.datetime.now()
    df = df.append(pd.DataFrame({'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))
    data=preprocessing_df(df)
    
    pre_arima=ARIMA_df(data.logDiff)
    data['pr_arima']=np.exp(float(pre_arima[-1])+df.log.shift(60))
    
    data=kalman_ma(data)
    data=strat_lr(data)
    data=classification_up_dn(data)
    
    data['predict_svm']=data.predict_svm_up+data.predict_svm_dn
    data['predict_lm']=data.predict_lm_up+data.predict_lm_dn
    #data['UD']=np.where(data.predict_svm+data.predict_lm>0,1,np.where(data.predict_svm+data.predict_lm<0,-1,0))  
    data['UD']=np.where(np.logical_and(data.predict_svm>0,data.predict_lm>0),1,np.where(np.logical_and(data.predict_svm<0,data.predict_lm<0),-1,0))  
    #data=data.dropna()   
    data['spread']=data.Close-data.pr_arima

    df_LSTM= data[['askPrice','askSize','bidPrice','bidSize','Close','spread','pr_arima','sigma']]
    pr,y=strat_LSTM(df_LSTM)
    UD=pr[-1]-y[-1]
    final=final.append(pd.DataFrame({'LSTM':float(UD)},index=[dt]))
    #output=data[['Close','price','km','pr_arima','pREG','pSVR','UD']]
    output=data[['Close','price','km','pr_arima','UD']]
    #output['spread']=data.pREG-data.SVR
    output['LSTM']=final.LSTM
       
    #plotting
    dt = datetime.datetime.now()
    s0.write({'x': str(dt)[11:-3], 'y': float(output['price'].tail(1))})
    #s1.write({'x': str(dt)[11:-3], 'y': float(output['km'].tail(1))})
    
    #trading

    #output['signal']=0
    #if output.Close.ix[-1]>output.km.ix[-1] and output.Close.ix[-1]>output.price.ix[-1] and output.LSTM.ix[-1]>output.LSTM.shift(1).ix[-1] and output.pr_arima.ix[-1]>output.pr_arima.shift(1).ix[-1]:
    #    output['signal']=1
        #print('long the market')
    #if output.Close.ix[-1]<output.km.ix[-1] and output.Close.ix[-1]<output.price.ix[-1] and output.LSTM.ix[-1]<output.LSTM.shift(1).ix[-1] and output.pr_arima.ix[-1]<output.pr_arima.shift(1).ix[-1]:
    #    output['signal']=-1
        #print('long the market')        
    
    #print(data.tail(1))
    print(output.tail(1))
    #print(output.signal.tail(1))
    #print(final)
    

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:58: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:74: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

                              Close       price          km    pr_arima  UD  \
2017-09-08 22:54:39.888967  246.845  246.840432  246.848182  246.855002   1   

                                LSTM  
2017-09-08 22:54:39.888967  0.318773  
                             Close   price          km    pr_arima  UD  \
2017-09-08 22:54:40.917290  246.85  246.85  246.848323  246.855002   1   

                                LSTM  
2017-09-08 22:54:40.917290  0.284741  
                             Close   price         km    pr_arima  UD     LSTM
2017-09-08 22:54:41.972271  246.85  246.85  246.84845  246.855002   0  0.02745
                              Close       price          km    pr_arima  UD  \
2017-09-08 22:54:43.038164  246.855  246.851646  246.848719  246.860002   0   

                                LSTM  
2017-09-08 22:54:43.038164  0.230136  
                              Close       price          km    pr_arima  UD  \
2017-09-08 22:54:44.666503  246.855  246.858182  246.849584  2

### Trading

In [ ]:
while True:
    iterations += 1
    string = socket.recv_string()
    sym, bidPrice,bidSize,askPrice,askSize = string.split()
    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))
    dt = datetime.datetime.now()
    df = df.append(pd.DataFrame({'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))
    data=preprocessing_df(df)
    
    pre_arima=ARIMA_df(data.logDiff)
    data['pr_arima']=np.exp(float(pre_arima[-1])+df.log.shift(60))
    
    data=kalman_ma(data)
    data=strat_lr(data)
    data=classification_up_dn(data)
    
    data['predict_svm']=data.predict_svm_up+data.predict_svm_dn
    data['predict_lm']=data.predict_lm_up+data.predict_lm_dn
    #data['UD']=np.where(data.predict_svm+data.predict_lm>0,1,np.where(data.predict_svm+data.predict_lm<0,-1,0))  
    data['UD']=np.where(np.logical_and(data.predict_svm>0,data.predict_lm>0),1,np.where(np.logical_and(data.predict_svm<0,data.predict_lm<0),-1,0))  
    #data=data.dropna()   
    data['spread']=data.Close-data.pr_arima

    df_LSTM= data[['askPrice','askSize','bidPrice','bidSize','Close','spread','pr_arima','sigma']]
    pr,y=strat_LSTM(df_LSTM)
    UD=pr[-1]-y[-1]
    final=final.append(pd.DataFrame({'LSTM':float(UD)},index=[dt]))
    #output=data[['Close','price','km','pr_arima','pREG','pSVR','UD']]
    output=data[['Close','price','km','pr_arima','UD']]
    #output['spread']=data.pREG-data.SVR
    output['LSTM']=final.LSTM
       
    #plotting
    dt = datetime.datetime.now()
    s0.write({'x': str(dt)[11:-3], 'y': float(output['price'].tail(1))})
    #s1.write({'x': str(dt)[11:-3], 'y': float(output['km'].tail(1))})
    
    #trading

    #output['signal']=0
    #if output.Close.ix[-1]>output.km.ix[-1] and output.Close.ix[-1]>output.price.ix[-1] and output.LSTM.ix[-1]>output.LSTM.shift(1).ix[-1] and output.pr_arima.ix[-1]>output.pr_arima.shift(1).ix[-1]:
    #    output['signal']=1
        #print('long the market')
    #if output.Close.ix[-1]<output.km.ix[-1] and output.Close.ix[-1]<output.price.ix[-1] and output.LSTM.ix[-1]<output.LSTM.shift(1).ix[-1] and output.pr_arima.ix[-1]<output.pr_arima.shift(1).ix[-1]:
    #    output['signal']=-1
        #print('long the market')        
    
    #print(data.tail(1))
    #print(output.tail(1))
    #print(output.signal.tail(1))
    #print(final)
    output.tail(1).to_csv('/home/octo/Dropbox/ml_output.txt', sep=',', encoding='utf-8')

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:58: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:74: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum